In [32]:
!pip install openai pandas

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.3 MB/s eta 0:00:00


In [31]:
!pip install --upgrade openai

In [30]:
!pip install chromadb

In [5]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:00


In [6]:
!pip install -U langchain-chroma

In [1]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
import chromadb
import openai
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI

# Set the API key manually in the environment
os.environ['OPENAI_API_KEY'] = 'sk-svcacct-QsCZJG1m7apYpNXmYDj73kS-eon0tOJ9Iy6XuNwWD8dH3IDn9QM_OgxlYV-O2-PlAET3BlbkFJ5LFfldkzioy0xrvORw_jOaQo2acpnCix3KApdmur4QmECN1t17iJA0nBrv6cSS74MA'

api_key='sk-svcacct-QsCZJG1m7apYpNXmYDj73kS-eon0tOJ9Iy6XuNwWD8dH3IDn9QM_OgxlYV-O2-PlAET3BlbkFJ5LFfldkzioy0xrvORw_jOaQo2acpnCix3KApdmur4QmECN1t17iJA0nBrv6cSS74MA'

# Access the API key
client_oai = OpenAI(api_key=os.environ['OPENAI_API_KEY'])


In [2]:
def check_openai_api_key(api_key):
    try:

        # Test the API by making a request to retrieve embeddings (a simple query)
        response = client_oai.embeddings.create(
            input="This is a test to check the API key",
            model="text-embedding-3-large"
        )

        print("API key is valid!")

    except client_oai.error.AuthenticationError:
        print("Invalid API key.")

    except Exception as e:
        print(f"An error occurred: {e}")

check_openai_api_key(api_key)

API key is valid!


In [3]:
# Load the Excel file into a Pandas DataFrame
df = pd.read_excel('df_with_metadata.xlsx')

In [4]:
# Preprocessing step to clean data by filling NaN values with 'No Data' and ensuring valid strings
df = df.fillna('No Data')  # Fill NaN values with 'No Data'
df = df.apply(lambda col: col.apply(lambda x: x if isinstance(x, str) and x.strip() else 'No Data'))
df = df.drop(columns=['Unnamed: 0'])

In [5]:
df

,File Name,Product Name,Supplier Name,Identification,Hazards identification,Composition/information on ingredients,First aid measures,Firefighting measures,Accidental release measures,Handling and storage,Exposure controls/personal protection,Physical and chemical properties,Stability and reactivity,Toxicological information,Ecological information,Disposal considerations,Transport information,Regulatory information,Other information,Document
0,0078gj-ghs11-div3-sds-4-aminopyridine-jubilant...,4-Aminopyridine,/UNDERTAKING.,IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF...,SECTION 2: HAZARD(S) IDENTIFICATION.\n2.1. Cla...,3: COMPOSITION/INFORMATION ON INGREDIENTS.\nCh...,4: FIRST AID MEASURES\n4.1. Description of fir...,5: FIRE-FIGHTING MEASURES\nFlash Point: 158℃ c...,6: ACCIDENTAL RELEASE MEASURES\n6.1. Personal ...,7: HANDLING AND STORAGE\n7.1. Precautions for ...,8: EXPOSURE CONTROLS / PERSONAL PROTECTION\n8....,SECTION 9 : PHYSICAL AND CHEMICAL PROPERTIES\n...,10: STABILITY AND REACTIVITY\n10.1 Reactivity\...,11: TOXICOLOGICAL INFORMATION\n11.1. Informati...,No Data,13: DISPOSAL CONSIDERATIONS\nWaste disposal re...,14: TRANSPORT INFORMATION\n· This substance is...,15: REGULATORY INFORMATION\n. European Union I...,16: OTHER INFORMATION\n(a) Compilation informa...,[{'page_content': 'IDENTIFICATION OF THE SUBST...
1,1-2-cyclohexanediamine-sds-fisher_2,"(+)-trans-1,2-Diaminocyclohexane",of the safety data sheet\nCompany Fisher Scien...,"Identification\nProduct Name\n(+)-trans-1,2-Di...",2. Hazard(s) identification\nClassification\nT...,3. Composition/Information on Ingredients\nCom...,4. First-aid measures\nEye Contact\nImmediate ...,5. Fire-fighting measures\nSuitable Extinguish...,6. Accidental release measures\nPersonal Preca...,7. Handling and storage\nHandling\nDo not brea...,8. Exposure controls / personal protection\nEx...,9. Physical and chemical properties\nPhysical ...,10. Stability and reactivity\nReactive Hazard\...,11. Toxicological information\nProduct Informa...,12. Ecological information\nEcotoxicity\nDo no...,13. Disposal considerations\nWaste Disposal Me...,14. Transport information\nPage 5 /7\n(+)-tran...,15. Regulatory information\nUnited States of A...,16. Other information\nRevision Date\n24-Dec-2...,[{'page_content': 'Identification\nProduct Nam...
2,2-aminopyrimidine-sds_2,2-Aminopyrimidine,/undertaking\nCompany : Sigma-Aldrich Inc. 305...,Identification of the substance/mixture and of...,2: Hazards identification\n2.1 Classification ...,3: Composition/information on ingredients\n3.1...,4: First aid measures\n4.1 Description of firs...,5: Firefighting measures\n5.1 Extinguishing me...,6: Accidental release measures\n6.1 Personal p...,7: Handling and storage\n7.1 Precautions for s...,8: Exposure controls/personal protection\n8.1 ...,9: Physical and chemical properties\n9.1 Infor...,10: Stability and reactivity\n10.1 Reactivity\...,11: Toxicological information\n11.1 Informatio...,12: Ecological information\n12.1 Toxicity\nNo ...,13: Disposal considerations\n13.1 Waste treatm...,14: Transport information\nDOT (US)\nNot dange...,15: Regulatory information\nSARA 302 Component...,16: Other information\nFurther information\nCo...,[{'page_content': 'Identification of the subst...
3,3-4-diaminopyridine-tci-sds-d1149_us_en_2,"3,4-Diaminopyridine",TCI America,"IDENTIFICATION\nProduct name:\n3,4-Diaminopyri...",2. HAZARD(S) IDENTIFICATION\nOSHA Haz Com: CFR...,3. COMPOSITION/INFORMATION ON INGREDIENTS\nSub...,4. FIRST-AID MEASURES\nDescription of first ai...,5. FIRE-FIGHTING MEASURES\nSuitable extinguish...,6. ACCIDENTAL RELEASE MEASURES\nPersonal preca...,7. HANDLING AND STORAGE\nPrecautions for safe ...,8. EXPOSURE CONTROLS / PERSONAL PROTECTION\nPa...,9. PHYSICAL AND CHEMICAL PROPERTIES\nPhysical ...,10. STABILITY AND REACTIVITY\nReactivity: No d...,11. TOXICOLOGICAL INFORMATION\nRTECS Number: U...,12. ECOLOGICAL INFORMATION\nEcotoxicity:\nFish...,13. DISPOSAL CONSIDERATIONS\nDisposal of produ...,14. TRANSPORT I

In [6]:
import os
import chromadb
from chromadb.config import Settings

In [7]:
# Define a writable directory for ChromaDB storage
path = "Chroma_db_storage"  # Use a relative path

# Create the directory if it does not exist
os.makedirs(path, exist_ok=True)

# Create a persistent client with the specified directory for storage
client = chromadb.PersistentClient(path=path)

print(f"Persistent client created with storage at: {os.path.abspath(path)}")

Persistent client created with storage at: C:\Users\sanje\Documents\WPI_Sem_3_Docs\GQP\working_directory\Chroma_db_storage


In [8]:
# Define the collection name
collection_name = "openai_sds_embeddings_metadata"

# Check if the collection already exists and delete it if it does
try:
    client.delete_collection(name=collection_name)
    print(f"Existing collection '{collection_name}' deleted.")
except Exception as e:
    print(f"No existing collection found or error deleting: {e}")

# Create a new collection
collection = client.create_collection(name=collection_name)

print(f"Collection '{collection_name}' created.")

No existing collection found or error deleting: Collection openai_sds_embeddings_metadata does not exist.
Collection 'openai_sds_embeddings_metadata' created.


In [11]:
df

,File Name,Product Name,Supplier Name,Identification,Hazards identification,Composition/information on ingredients,First aid measures,Firefighting measures,Accidental release measures,Handling and storage,Exposure controls/personal protection,Physical and chemical properties,Stability and reactivity,Toxicological information,Ecological information,Disposal considerations,Transport information,Regulatory information,Other information,Document
0,0078gj-ghs11-div3-sds-4-aminopyridine-jubilant...,4-Aminopyridine,/UNDERTAKING.,IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF...,SECTION 2: HAZARD(S) IDENTIFICATION.\n2.1. Cla...,3: COMPOSITION/INFORMATION ON INGREDIENTS.\nCh...,4: FIRST AID MEASURES\n4.1. Description of fir...,5: FIRE-FIGHTING MEASURES\nFlash Point: 158℃ c...,6: ACCIDENTAL RELEASE MEASURES\n6.1. Personal ...,7: HANDLING AND STORAGE\n7.1. Precautions for ...,8: EXPOSURE CONTROLS / PERSONAL PROTECTION\n8....,SECTION 9 : PHYSICAL AND CHEMICAL PROPERTIES\n...,10: STABILITY AND REACTIVITY\n10.1 Reactivity\...,11: TOXICOLOGICAL INFORMATION\n11.1. Informati...,No Data,13: DISPOSAL CONSIDERATIONS\nWaste disposal re...,14: TRANSPORT INFORMATION\n· This substance is...,15: REGULATORY INFORMATION\n. European Union I...,16: OTHER INFORMATION\n(a) Compilation informa...,[{'page_content': 'IDENTIFICATION OF THE SUBST...
1,1-2-cyclohexanediamine-sds-fisher_2,"(+)-trans-1,2-Diaminocyclohexane",of the safety data sheet\nCompany Fisher Scien...,"Identification\nProduct Name\n(+)-trans-1,2-Di...",2. Hazard(s) identification\nClassification\nT...,3. Composition/Information on Ingredients\nCom...,4. First-aid measures\nEye Contact\nImmediate ...,5. Fire-fighting measures\nSuitable Extinguish...,6. Accidental release measures\nPersonal Preca...,7. Handling and storage\nHandling\nDo not brea...,8. Exposure controls / personal protection\nEx...,9. Physical and chemical properties\nPhysical ...,10. Stability and reactivity\nReactive Hazard\...,11. Toxicological information\nProduct Informa...,12. Ecological information\nEcotoxicity\nDo no...,13. Disposal considerations\nWaste Disposal Me...,14. Transport information\nPage 5 /7\n(+)-tran...,15. Regulatory information\nUnited States of A...,16. Other information\nRevision Date\n24-Dec-2...,[{'page_content': 'Identification\nProduct Nam...
2,2-aminopyrimidine-sds_2,2-Aminopyrimidine,/undertaking\nCompany : Sigma-Aldrich Inc. 305...,Identification of the substance/mixture and of...,2: Hazards identification\n2.1 Classification ...,3: Composition/information on ingredients\n3.1...,4: First aid measures\n4.1 Description of firs...,5: Firefighting measures\n5.1 Extinguishing me...,6: Accidental release measures\n6.1 Personal p...,7: Handling and storage\n7.1 Precautions for s...,8: Exposure controls/personal protection\n8.1 ...,9: Physical and chemical properties\n9.1 Infor...,10: Stability and reactivity\n10.1 Reactivity\...,11: Toxicological information\n11.1 Informatio...,12: Ecological information\n12.1 Toxicity\nNo ...,13: Disposal considerations\n13.1 Waste treatm...,14: Transport information\nDOT (US)\nNot dange...,15: Regulatory information\nSARA 302 Component...,16: Other information\nFurther information\nCo...,[{'page_content': 'Identification of the subst...
3,3-4-diaminopyridine-tci-sds-d1149_us_en_2,"3,4-Diaminopyridine",TCI America,"IDENTIFICATION\nProduct name:\n3,4-Diaminopyri...",2. HAZARD(S) IDENTIFICATION\nOSHA Haz Com: CFR...,3. COMPOSITION/INFORMATION ON INGREDIENTS\nSub...,4. FIRST-AID MEASURES\nDescription of first ai...,5. FIRE-FIGHTING MEASURES\nSuitable extinguish...,6. ACCIDENTAL RELEASE MEASURES\nPersonal preca...,7. HANDLING AND STORAGE\nPrecautions for safe ...,8. EXPOSURE CONTROLS / PERSONAL PROTECTION\nPa...,9. PHYSICAL AND CHEMICAL PROPERTIES\nPhysical ...,10. STABILITY AND REACTIVITY\nReactivity: No d...,11. TOXICOLOGICAL INFORMATION\nRTECS Number: U...,12. ECOLOGICAL INFORMATION\nEcotoxicity:\nFish...,13. DISPOSAL CONSIDERATIONS\nDisposal of produ...,14. TRANSPORT I

In [12]:
# Define the get_embedding function
def get_embedding(text, model="text-embedding-3-large"):
    client_oai = OpenAI()

    try:
        if isinstance(text, str) and text.strip():
            text = text.replace("\n", " ")
            tokens = len(text.split())
            if tokens > 8192:
                text = " ".join(text.split()[:8192])

            response = client_oai.embeddings.create(input=[text], model=model)
            return response.data[0].embedding
        return None
    except Exception as e:
        print(f"Embedding generation error: {e}")
        return None



In [13]:
def store_sds_documents_to_chromadb(df):
    successful_stores = 0
    failed_stores = 0

    for index, row in df.iterrows():
        # Extract the document content and metadata directly from the "Document" column
        document_content = row.get("Document", None)

        if document_content:
            # Generate embedding for the document content
            embedding = get_embedding(document_content)

            if embedding:
                unique_id = f"{index}_document"

                # Assume document content already includes the metadata we want
                # Store embedding with the document content and its existing metadata
                collection.add(
                    embeddings=[embedding],
                    documents=[document_content],  # Store the full document content directly
                    ids=[unique_id],
                    metadatas=[document_content['metadata']]  # Use the embedded metadata directly
                )
                successful_stores += 1
                print(f"Stored embedding for document {unique_id}")
            else:
                failed_stores += 1

    print(f"\nEmbedding storage completed with {successful_stores} successes and {failed_stores} failures")

# Call the function on the DataFrame
store_sds_documents_to_chromadb(df)


TypeError: string indices must be integers, not 'str'

In [14]:
# List all collections in the persistent client
collections = client.list_collections()
print(f"Existing collections: {[col.name for col in collections]}")

Existing collections: ['openai_sds_embeddings_metadata']


In [21]:
os.environ['OPENAI_API_KEY'] = 'sk-svcacct-QsCZJG1m7apYpNXmYDj73kS-eon0tOJ9Iy6XuNwWD8dH3IDn9QM_OgxlYV-O2-PlAET3BlbkFJ5LFfldkzioy0xrvORw_jOaQo2acpnCix3KApdmur4QmECN1t17iJA0nBrv6cSS74MA'


In [15]:
# Function to view collection items
def view_collection_items(collection, num_items=10):
    """
    View a sample of items from the ChromaDB collection.

    Parameters:
    - collection: The ChromaDB collection object.
    - num_items: Number of items to retrieve for display (default is 10).
    """
    try:
        # Get the items from the collection (limited by num_items)
        results = collection.get(
            ids=None,  # Retrieve all items (can be limited)
            limit=num_items,  # Limit to num_items for display
            include=['embeddings', 'metadatas', 'documents']  # What to include in the results
        )

        # Print the retrieved items
        for idx, metadata in enumerate(results['metadatas']):
            print(f"\nItem {idx + 1}:")
            print(f"ID: {results['ids'][idx]}")
            print(f"Metadata: {metadata}")
            print(f"Document: {results['documents'][idx]}")
            print(f"Embedding Length: {len(results['embeddings'][idx])} (First 5 values: {results['embeddings'][idx][:5]})")

        print(f"\nTotal items retrieved: {len(results['ids'])}")

    except Exception as e:
        print(f"Error retrieving collection items: {e}")

In [16]:
# Function to filter collection items by content
def filter_collection_by_document_content(collection, document_filter, num_items=10):
    """
    Filter and view items from the ChromaDB collection based on document content.

    Parameters:
    - collection: The ChromaDB collection object.
    - document_filter: Dictionary to filter by document content.
    - num_items: Number of items to retrieve for display (default is 10).
    """
    try:
        # Get the filtered items from the collection (limited by num_items)
        results = collection.get(
            ids=None,  # Retrieve all matching items
            limit=num_items,  # Limit to num_items for display
            where_document=document_filter,  # Filter based on document content
            include=['embeddings', 'metadatas', 'documents']  # What to include in the results
        )

        # Print the retrieved items
        for idx, metadata in enumerate(results['metadatas']):
            print(f"\nItem {idx + 1}:")
            print(f"ID: {results['ids'][idx]}")
            print(f"Metadata: {metadata}")
            print(f"Document: {results['documents'][idx]}")
            print(f"Embedding Length: {len(results['embeddings'][idx])} (First 5 values: {results['embeddings'][idx][:5]})")

        print(f"\nTotal items retrieved: {len(results['ids'])}")

    except Exception as e:
        print(f"Error retrieving collection items: {e}")

In [17]:
# Function to filter collection items by metadata and content
def filter_collection_by_metadata_and_content(collection, metadata_filter, document_content_filters, num_items=10):
    """
    Filter and view items from the ChromaDB collection based on metadata and document content.

    Parameters:
    - collection: The ChromaDB collection object.
    - metadata_filter: Dictionary to filter by metadata.
    - document_content_filters: Dictionary to filter by multiple conditions in document content.
    - num_items: Number of items to retrieve for display (default is 10).
    """
    try:
        # Get the filtered items from the collection (limited by num_items)
        results = collection.get(
            ids=None,  # Retrieve all matching items
            limit=num_items,  # Limit to num_items for display
            where=metadata_filter,  # Filter based on metadata
            where_document=document_content_filters,  # Filter based on document content
            include=['embeddings', 'metadatas', 'documents']  # What to include in the results
        )

        # Print the retrieved items
        for idx, metadata in enumerate(results['metadatas']):
            print(f"\nItem {idx + 1}:")
            print(f"ID: {results['ids'][idx]}")
            print(f"Metadata: {metadata}")
            print(f"Document: {results['documents'][idx]}")
            print(f"Embedding Length: {len(results['embeddings'][idx])} (First 5 values: {results['embeddings'][idx][:5]})")

        print(f"\nTotal items retrieved: {len(results['ids'])}")

    except Exception as e:
        print(f"Error retrieving collection items: {e}")

In [18]:
collection_name = "openai_sds_embeddings_metadata"
collection = client.get_collection(name=collection_name)

# Usage: Return a given number of items
view_collection_items(collection, num_items=18)


Item 1:
ID: 0_document
Metadata: {'file_name': '0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2', 'product_name': '4-Aminopyridine', 'supplier': '/UNDERTAKING.'}
Document: [{'page_content': 'IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF THE COMPANY/UNDERTAKING.\n1.1. Product identifier\nPRODUCT NAME\n: 4-Aminopyridine\nCAS RN\n: 504-24-5\nEC#\n: 207-987-9\nSYNONYMS\n: y-pyridylamine; Gamma-aminopyridine; m-aminopyridine; amino-4-pyridine\nSYSTEMATIC NAME MOLECULAR FORMULA STRUCTURAL FORMULA\n: 4-Aminopyridine, 4-Pyridinamine, 4-Pyridylamine, Pyridine : C5H6N2 :\nNH2\nN\n1.2. Relevant identified uses of the substance or mixture and uses advised against\n1.2.1. Relevant identified uses 4-Aminopyridine is used as an Intermediate, Bird Repellant avicide and in Medication.\nUses advised against: None\n1.3. Details of the supplier of the safety data sheet Jubilant Ingrevia Limited FACTORY & REGISTERED OFFICE: Jubilant Ingrevia Limited., Bhartiagram, Gajraula, District: Amroha, Uttar P

In [19]:
# Example usage:
document_filter = {
    "$contains": "Serious Eye Damage"  # Filtering documents that contain the phrase "Serious Eye Damage"
}

filter_collection_by_document_content(collection, document_filter)


Item 1:
ID: 0_document
Metadata: {'file_name': '0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2', 'product_name': '4-Aminopyridine', 'supplier': '/UNDERTAKING.'}
Document: [{'page_content': 'IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF THE COMPANY/UNDERTAKING.\n1.1. Product identifier\nPRODUCT NAME\n: 4-Aminopyridine\nCAS RN\n: 504-24-5\nEC#\n: 207-987-9\nSYNONYMS\n: y-pyridylamine; Gamma-aminopyridine; m-aminopyridine; amino-4-pyridine\nSYSTEMATIC NAME MOLECULAR FORMULA STRUCTURAL FORMULA\n: 4-Aminopyridine, 4-Pyridinamine, 4-Pyridylamine, Pyridine : C5H6N2 :\nNH2\nN\n1.2. Relevant identified uses of the substance or mixture and uses advised against\n1.2.1. Relevant identified uses 4-Aminopyridine is used as an Intermediate, Bird Repellant avicide and in Medication.\nUses advised against: None\n1.3. Details of the supplier of the safety data sheet Jubilant Ingrevia Limited FACTORY & REGISTERED OFFICE: Jubilant Ingrevia Limited., Bhartiagram, Gajraula, District: Amroha, Uttar P

In [20]:
# Example usage:
document_filter = {
    "$contains": "Skin Corrosion"  # Filtering documents that contain the phrase "Skin Corrosion"
}

filter_collection_by_document_content(collection, document_filter)


Item 1:
ID: 0_document
Metadata: {'file_name': '0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2', 'product_name': '4-Aminopyridine', 'supplier': '/UNDERTAKING.'}
Document: [{'page_content': 'IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF THE COMPANY/UNDERTAKING.\n1.1. Product identifier\nPRODUCT NAME\n: 4-Aminopyridine\nCAS RN\n: 504-24-5\nEC#\n: 207-987-9\nSYNONYMS\n: y-pyridylamine; Gamma-aminopyridine; m-aminopyridine; amino-4-pyridine\nSYSTEMATIC NAME MOLECULAR FORMULA STRUCTURAL FORMULA\n: 4-Aminopyridine, 4-Pyridinamine, 4-Pyridylamine, Pyridine : C5H6N2 :\nNH2\nN\n1.2. Relevant identified uses of the substance or mixture and uses advised against\n1.2.1. Relevant identified uses 4-Aminopyridine is used as an Intermediate, Bird Repellant avicide and in Medication.\nUses advised against: None\n1.3. Details of the supplier of the safety data sheet Jubilant Ingrevia Limited FACTORY & REGISTERED OFFICE: Jubilant Ingrevia Limited., Bhartiagram, Gajraula, District: Amroha, Uttar P

In [21]:
# Example usage:
document_filter = {
    "$and": [  # Use the $and operator to combine multiple conditions
        {"$contains": "Skin Corrosion"},  # Condition for "Skin Corrosion"
        {"$contains": "Serious Eye Damage"}  # Condition for "Serious Eye Damage"
    ]
}

filter_collection_by_document_content(collection, document_filter)


Item 1:
ID: 0_document
Metadata: {'file_name': '0078gj-ghs11-div3-sds-4-aminopyridine-jubilant-sds_2', 'product_name': '4-Aminopyridine', 'supplier': '/UNDERTAKING.'}
Document: [{'page_content': 'IDENTIFICATION OF THE SUBSTANCE/MIXTURE AND OF THE COMPANY/UNDERTAKING.\n1.1. Product identifier\nPRODUCT NAME\n: 4-Aminopyridine\nCAS RN\n: 504-24-5\nEC#\n: 207-987-9\nSYNONYMS\n: y-pyridylamine; Gamma-aminopyridine; m-aminopyridine; amino-4-pyridine\nSYSTEMATIC NAME MOLECULAR FORMULA STRUCTURAL FORMULA\n: 4-Aminopyridine, 4-Pyridinamine, 4-Pyridylamine, Pyridine : C5H6N2 :\nNH2\nN\n1.2. Relevant identified uses of the substance or mixture and uses advised against\n1.2.1. Relevant identified uses 4-Aminopyridine is used as an Intermediate, Bird Repellant avicide and in Medication.\nUses advised against: None\n1.3. Details of the supplier of the safety data sheet Jubilant Ingrevia Limited FACTORY & REGISTERED OFFICE: Jubilant Ingrevia Limited., Bhartiagram, Gajraula, District: Amroha, Uttar P

In [22]:
# Example usage:
metadata_filter = {
    "product_name": "(+)-trans-1,2-Diaminocyclohexane"  # Filter by specific product name
}

document_filter = {
    "$and": [  # Use the $and operator to combine multiple conditions
        {"$contains": "Skin Corrosion"},  # Condition for "Skin Corrosion"
        {"$contains": "Serious Eye Damage"}  # Condition for "Serious Eye Damage"
    ]
}

filter_collection_by_metadata_and_content(collection, metadata_filter, document_filter)


Item 1:
ID: 1_document
Metadata: {'file_name': '1-2-cyclohexanediamine-sds-fisher_2', 'product_name': '(+)-trans-1,2-Diaminocyclohexane', 'supplier': 'of the safety data sheet\nCompany Fisher Scientific Company One Reagent Lane Fair Lawn, NJ 07410 Tel: (201) 796-7100 Emergency Telephone Number For information US call: 001-800-ACROS-01 / Europe call: +32 14 57 52 11 Emergency Number US:001-201-796-7100 / Europe: +32 14 57 52 99 CHEMTREC Tel. No.US:001-800-424-9300 / Europe:001-703-527-3887 Acros Organics One Reagent Lane Fair Lawn, NJ 07410'}
Document: [{'page_content': 'Identification\nProduct Name\n(+)-trans-1,2-Diaminocyclohexane\nCat No. :\nAC248670000; AC248670100; AC248670500; AC248672500\nCAS No\n1121-22-8\nSynonyms\n(#1)-trans-1,2-Cyclohexanediamine\nRecommended Use\nLaboratory chemicals.\nUses advised against\nFood, drug, pesticide or biocidal product use.\nDetails of the supplier of the safety data sheet\nCompany\nFisher Scientific Company\nOne Reagent Lane\nFair Lawn, NJ 074